<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/notebooks/US_Census_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal of Notebook - Read US Census data for 2001 -2020

In [ ]:
try:
    from google.colab import drive

    drive.mount('./drive/')

    %cd drive/Shareddrives/Data606_Energy

except:
    print("No Colab Environment")

Mounted at ./drive/
/content/drive/Shareddrives/Data606_Energy


In [ ]:
import sys
import pandas as pd
import numpy as np
import glob
from tqdm.autonotebook import tqdm
import plotly.express as px
sys.path.append("helpers/")

from energygrid import EGRID

from helper_functions import write_csv,combine_like_files

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [ ]:
import json

with open("resources/us_state_abbreviation.json") as f:

    us_states = json.loads(f.read())

In [ ]:
df_state_names = pd.DataFrame([us_states.keys(),us_states.values()]).T.rename(columns={0:"state_name",1:"abbrv"}).query('state_name != "Puerto Rico"')

In [ ]:
df_state_names.head()

,state_name,abbrv
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [ ]:
url_2000_2009 = "https://www2.census.gov/programs-surveys/popest/tables/2000-2010/intercensal/state/st-est00int-01.xls"

df_2000_2009_data = (pd.read_excel(url_2000_2009,header=[3])
                .rename(columns={"Unnamed: 0":"state_name"})
                .drop(columns=["Unnamed: 1","Unnamed: 12","Unnamed: 13"])
                )

df_2000_2009_data

,state_name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,United States,282162411.0,284968955.0,287625193.0,290107933.0,292805298.0,295516599.0,298379912.0,301231207.0,304093966.0,306771529.0
1,Northeast,53666295.0,53915522.0,54143915.0,54334453.0,54423533.0,54451230.0,54522659.0,54653362.0,54875926.0,55133101.0
2,Midwest,64491431.0,64776531.0,65018293.0,65276954.0,65532305.0,65751872.0,66028555.0,66293689.0,66523935.0,66748437.0
3,South,100565549.0,101849575.0,103150787.0,104380188.0,105883977.0,107479771.0,109076933.0,110688742.0,112184930.0,113548615.0
4,West,63439136.0,64427327.0,65312198.0,66116338.0,66965483.0,67833726.0,68751765.0,69595414.0,70509175.0,71341376.0
...,...,...,...,...,...,...,...,...,...,...,...
61,Note: See Geographic Terms and Definitions at ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Table 1. Intercensal Estimates of the Resident...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2000_2009_data.head(10)

,state_name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,United States,282162411.0,284968955.0,287625193.0,290107933.0,292805298.0,295516599.0,298379912.0,301231207.0,304093966.0,306771529.0
1,Northeast,53666295.0,53915522.0,54143915.0,54334453.0,54423533.0,54451230.0,54522659.0,54653362.0,54875926.0,55133101.0
2,Midwest,64491431.0,64776531.0,65018293.0,65276954.0,65532305.0,65751872.0,66028555.0,66293689.0,66523935.0,66748437.0
3,South,100565549.0,101849575.0,103150787.0,104380188.0,105883977.0,107479771.0,109076933.0,110688742.0,112184930.0,113548615.0
4,West,63439136.0,64427327.0,65312198.0,66116338.0,66965483.0,67833726.0,68751765.0,69595414.0,70509175.0,71341376.0
5,.Alabama,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
6,.Alaska,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0
7,.Arizona,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0
8,.Arkansas,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0
9,.California,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0


In [ ]:
df_2000_2009_data["state_name"] = df_2000_2009_data["state_name"].str.strip(".")

df_2000_2009 = df_state_names.merge(df_2000_2009_data,on="state_name",how="left").dropna()

df_2000_2009.head()

,state_name,abbrv,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,AL,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
1,Alaska,AK,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0
2,Arizona,AZ,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0
3,Arkansas,AR,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0
4,California,CA,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0


In [ ]:
# Not used anymore
url_2010_2019 = "https://www2.census.gov/programs-surveys/popest/tables/2010-2020/state/totals/nst-est2020.xlsx"
df_2010_2019_data = pd.read_excel(url_2010_2019,header=[3]).rename(columns={"Unnamed: 0":"state_name"}).drop(columns=["Census","Estimates Base","April 1","July 1"])
df_2010_2019_data["state_name"] = df_2010_2019_data["state_name"].str.strip(".")

In [ ]:
df_2010_2019_data.query('state_name == "Alabama"')

,state_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
5,Alabama,4785514.0,4799642.0,4816632.0,4831586.0,4843737.0,4854803.0,4866824.0,4877989.0,4891628.0,4907965.0


In [ ]:
df_2010_2020 = pd.read_csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/state/totals/nst-est2020.csv")

df_2010_2020_states = df_state_names.merge(df_2010_2020,left_on="state_name",right_on="NAME").drop(columns=["POPESTIMATE042020"])

In [ ]:
keep_cols = [col for col in df_2010_2020_states.columns if col.startswith("POPESTIMATE") or "state" in col or "abbrv" in col]

df_2010_2020_states_trim = df_2010_2020_states[keep_cols]

In [ ]:
df_2010_2020_states_trim

,state_name,abbrv,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020
0,Alabama,AL,4785514,4799642,4816632,4831586,4843737,4854803,4866824,4877989,4891628,4907965,4921532
1,Alaska,AK,713982,722349,730810,737626,737075,738430,742575,740983,736624,733603,731158
2,Arizona,AZ,6407342,6473416,6556344,6634690,6732873,6832810,6944767,7048088,7164228,7291843,7421401
3,Arkansas,AR,2921998,2941038,2952876,2960459,2968759,2979732,2991815,3003855,3012161,3020985,3030522
4,California,CA,37319550,37636311,37944551,38253768,38586706,38904296,39149186,39337785,39437463,39437610,39368078
5,Colorado,CO,5047539,5121900,5193660,5270774,5352637,5454328,5543844,5617421,5697155,5758486,5807719
6,Connecticut,CT,3579173,3588632,3595211,3595792,3595697,3588561,3579830,3575324,3574561,3566022,3557006
7,Delaware,DE,899647,907590,915518,924062,933131,942065,949989,957942,966985,976668,986809
8,Florida,FL,18846143,19055607,19302016,19551678,19853880,20219111,20627237,20977089,21254926,21492056,21733312
9,Georgia,GA,9712209,9803630,9903580,9975592,10071204,10183353,10308442,10417031,10519389,10628020,10710017


In [ ]:
df_2010_2020_states_trim.columns = [col[-4:].strip() if col.startswith("POPESTIMATE") else col for col in df_2010_2020_states_trim]

df_2010_2020_states_trim.query('state_name=="Alabama"')

,state_name,abbrv,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alabama,AL,4785514,4799642,4816632,4831586,4843737,4854803,4866824,4877989,4891628,4907965,4921532


In [ ]:
df_census_2000_2020 = df_2000_2009.merge(df_2010_2020_states_trim,on=["state_name","abbrv"],how="left")

In [ ]:
df_census_2000_2020.head()

,state_name,abbrv,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alabama,AL,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0,4785514,4799642,4816632,4831586,4843737,4854803,4866824,4877989,4891628,4907965,4921532
1,Alaska,AK,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0,713982,722349,730810,737626,737075,738430,742575,740983,736624,733603,731158
2,Arizona,AZ,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0,6407342,6473416,6556344,6634690,6732873,6832810,6944767,7048088,7164228,7291843,7421401
3,Arkansas,AR,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0,2921998,2941038,2952876,2960459,2968759,2979732,2991815,3003855,3012161,3020985,3030522
4,California,CA,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0,37319550,37636311,37944551,38253768,38586706,38904296,39149186,39337785,39437463,39437610,39368078


In [ ]:
df_complete = (df_census_2000_2020.melt(id_vars=["state_name","abbrv"], var_name="year", value_name="state_population")
                .reset_index(drop=True)
                )

df_complete.head()

,state_name,abbrv,year,state_population
0,Alabama,AL,2000,4452173.0
1,Alaska,AK,2000,627963.0
2,Arizona,AZ,2000,5160586.0
3,Arkansas,AR,2000,2678588.0
4,California,CA,2000,33987977.0


In [ ]:
df_complete["year"].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020'], dtype=object)

In [ ]:
df_complete.groupby("year").count()

,state_name,abbrv,state_population
year,,,
2000,51,51,51
2001,51,51,51
2002,51,51,51
2003,51,51,51
2004,51,51,51
2005,51,51,51
2006,51,51,51
2007,51,51,51
2008,51,51,51


In [ ]:
#df_complete.drop(columns=["state_name"]).to_csv("data/percapita/us_census/us_census_2000_2020.csv",index=False)

In [ ]:
import plotly.express as px

px.line(df_complete,x="year",y="state_population",color="state_name",title="US State Population")